In [1]:
import pandas as pd
import requests
import psycopg2
import os
import sqlalchemy
import json
import numpy as np

In [2]:
# Database credentials
ENDPOINT = 'datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com'
DB_NAME = 'datalakeTwitter'
USERNAME = 'nadine'
PASSWORD = '30101995'

In [3]:
# connecting to the database
try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
        
try:
    cur = conn.cursor()
    engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

    
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
        
# Create sqlalchemy engine
engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

host=datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com dbname=datalakeTwitter user=nadine password=30101995


In [4]:
conn.set_session(autocommit=True)

In [5]:
#fetching data from datalake and save it as variable
data = engine.execute('Select * FROM tasty_twitter;').fetchall()

In [6]:
#creating a pandas dataframe for further steps
df = pd.DataFrame(data)

In [7]:
# creating new columns
df['url_short'] = ''
df['displayed_url'] = ''
df['expanded_url'] = ''
df['symbols'] = ''
#df['hashtags'] = ''
#df['user_mentions'] = ''

In [8]:
# fetch the link out of the additional_information and appends it to the newly created colums above
for i, row in df.iterrows(): 
    urls = (row.additional_information.get('urls'))
    #df['media'].iloc[i] = row.additional_information.get('media')
    df['symbols'].iloc[i] = row.additional_information.get('symbols')
    #df['hashtags'].iloc[i] = row.additional_information.get('hashtags')
    #df['user_mentions'].iloc[i] = row.additional_information.get('user_mentions')
    if not urls:
        df['url_short'].iloc[i] = np.nan
        df['displayed_url'].iloc[i] = np.nan
        df['expanded_url'].iloc[i] = np.nan
    else:
        for url in urls:
            df['url_short'].iloc[i]= url.get('url')
            df['displayed_url'].iloc[i]= url.get('display_url')
            df['expanded_url'].iloc[i]= url.get('expanded_url')

C:\Users\nadin\AppData\Local\Temp\ipykernel_23840\867637988.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbols'].iloc[i] = row.additional_information.get('symbols')
C:\Users\nadin\AppData\Local\Temp\ipykernel_23840\867637988.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url_short'].iloc[i] = np.nan
C:\Users\nadin\AppData\Local\Temp\ipykernel_23840\867637988.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[

In [9]:
# filling in empty values with na and dropping column additional_information
df = df.fillna(value=np.nan)
df = df.drop(['additional_information'], axis = 1)
df.symbols = df.symbols.apply(lambda y: np.nan if len(y)==0 else y)
#df.hashtags = df.hashtags.apply(lambda y: np.nan if len(y)==0 else y)
#df.user_mentions = df.user_mentions.apply(lambda y: np.nan if len(y)==0 else y)
#display(df)

In [10]:
# in the following steps the recipe name is extraxted out of the link
# first a column with 0 or 1 is created to indicate if the tweet has a recipe in it
recipe_link_list = []

for item in df['tweet']:
    if 'Shop the recipe!' in item:
        recipe_link_list.append(1)
    else:
        recipe_link_list.append(0)
df['recipe_link'] = recipe_link_list

In [11]:
# the type is adapted and a sub dataframe is created for the furhter steps
df['expanded_url'] = df['expanded_url'].astype(str)
df_recipes_yes = df[df['recipe_link'] == 1]

In [12]:
# the link is splited and joined to extract the recipe name which is then appended to a list
recipe_name = []
for recipe in df_recipes_yes['expanded_url']:
    test = recipe.split('/')
    if len(test) >= 4:
        test_split= test[4].split('-')
        test = " ".join(test_split)
        recipe_name.append(test)

In [13]:
# out of the recipe name an iterater is generated to append the recipe name to the rows that have a recipe
iter_recipe_name = iter(recipe_name)
df_recipe_name = []

for element in df['recipe_link']:
    if element == 1:
        df_recipe_name.append(next(iter_recipe_name))
    else:
        df_recipe_name.append(np.nan)

In [14]:
# the created list is then appended to the dataframe
df['recipe_name'] = df_recipe_name

In [16]:
#display(df)

In [18]:
conn.set_session(autocommit=True)
cur.execute("CREATE TABLE IF NOT EXISTS tasty_twitter_clean (tweet TEXT, created DATE, favorites INT, count INT, url_short TEXT, displayed_url TEXT, expanded_url TEXT, symbols TEXT, recipe_link CHAR, recipe_name TEXT);")

In [19]:
#adding the df into the database
df.to_sql('tasty_twitter_clean', engine, if_exists = 'replace', index = False)

200

In [21]:
#close the connections
conn.close()
#engine.close()